In [ ]:
import os
import glob

import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave

In [ ]:
data_root = "/home/azureuser/cloudfiles/code/Users/jordan.pierce/Data/Guam_Saipan/3361/"
image_path = data_root + "images/"
label_path = data_root + "CNet_Annotations.csv"
patch_path = data_root + "patches/"

os.makedirs(patch_path, exist_ok=True)

df = pd.read_csv(label_path)
imported_df = df[['Name', 'Row', 'Column', 'Label']]

In [ ]:
# sampled_df = pd.DataFrame()

# for class_category in imported_df['Label'].unique():

#     try:
#         class_sample = imported_df[imported_df['Label'] == class_category].sample(n=100)
#     except:
#         class_sample = imported_df[imported_df['Label'] == class_category].sample(n=100, replace=True)
        
#     sampled_df = pd.concat((sampled_df, class_sample))

# sampled_df['Label'].value_counts()

In [ ]:
import concurrent.futures


def crop_patch(image, y, x, size=112):
    """Given an image, and a Y, X location, this function will extract the patch. 
        Default patch size is 112 x 112"""

    patch = image[abs(size - y) : abs(size + y), abs(size - x) : abs(size + x), :]
    
    return patch
    

def process_row(r):

    patch_name = "_" + r['Label'] + "_" + str(r['Row']) + "_" + str(r["Column"]) + ".png"
    patch_name = r["Name"].split(".")[0] + patch_name
    patch_name = patch_path + patch_name

    image = image_path + r['Name']
    img = imread(image)
    patch = crop_patch(img, r['Row'], r['Column'])

    try:
        if not os.path.exists(patch_name):
            imsave(fname=patch_name, arr=patch)
        return [r['Name'], r['Label'], patch_name]

    except Exception as e:
        print(e)
        pass

    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    
    futures = []

    for i, r in imported_df.iterrows():

        futures.append(executor.submit(process_row, r))


    updated_imported_df = []

    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result is not None:
            updated_imported_df.append(result)

    updated_imported_df = pd.DataFrame(updated_imported_df, columns=['Image', 'Label', 'Patch_Name'])
    updated_imported_df.to_csv(data_root + "Updated_CNet_Annotations.csv")

In [ ]:
updated_imported_df.shape